In [ ]:
source("~/sc-online/extraGenes.R")
source("~/sc-online/myExpSetCreatorFn.R")
source("~/sc-online/utils.R")
source("~/sc-online/labelTransfer.R")

library(caret)
library("DropletUtils")
library(dplyr)
library(ggplot2)
library(grid)
library(gridExtra)
library(gtable)
library(lisi)
library(Matrix)
library(patchwork)
library(pheatmap)
library(RColorBrewer)
library(rhdf5)
library(rlang)

library(Seurat)
library(tidyr)
library(xml2)

library(viridis)
library(viridisLite)


MAKE_AND_SAVE_FILTERED_SCE_RDS = TRUE
SAVE_RAW_RDS = TRUE
SAVE_MOLECULE_INFO_RDS = FALSE
SAVE_CELLBENDER_RDS = TRUE

ORGANISM = "human"
MIN_EXP_CELLS=0

BASE_PATH = "/mnt/accessory/seq_data"
DATA_ING_DIRNAME = "ingested_data"
MOLECULE_INFO_BASENAME = "cr_outs/molecule_info.h5"
RAW_COUNTS_BASENAME = "cr_outs/raw_feature_bc_matrix.h5"
FILTERED_COUNTS_BASENAME = "cr_outs/filtered_feature_bc_matrix.h5"
VIREO_DONOR_IDS_BASENAME = "vireo_outs/donor_list/donor_ids.tsv"


SUPPLEMENTARY_METADATA_PATH = "/home/ferris/pd_lib_info_20240301.tsv"
FPR = 0.01

In [ ]:
CALICO_DIRS_LONG = readLines("~/calico-libs-long.txt")
GTEX_DIRS_LONG = readLines("~/gtex-libs-long.txt")
DIRS_TO_READ_LONG = c(CALICO_DIRS_LONG, GTEX_DIRS_LONG)

# Extract text after the first underscore in each directory name if it begins with a date
# Function to extract text based on the pattern
extract_text <- function(dir) {
    # Regular expression to match one or more digits followed by an underscore
    pattern <- "^[0-9]+_"
  
    # If the directory name matches the pattern, extract the part after the underscore
    # Otherwise, return the whole directory name
    if (grepl(pattern, dir)) {
        return(sub(pattern, "", dir))
    } else {
        return(dir)
    }
}


createMetaDataFromDGCMatrix=function(
    expt_df,
    umi_dgc, 
    mol_dgc,
    donor_ids){
    
    # INPUTS
    # expt_df, a row of a dataframe 
    # umi_dgc, a dgc matrix derived from a Read 10X filtered feature matrix h5 file
    # mol_dgc, a dgc matrix derived from a Read 10X molecule info h5 file
    # donor_ids, a dataframe derived from vireo's donor assignment tsv file

    # OUTPUTS 
    #a dataframe, whose rownames are the colnames of umi_dgc
    
    # output containing columns:
    #   nUMI: total number of UMIs within a cell
    #   nGene: number of unique genes expressed within a cell
    #   nRead: number of total reads within a cell
    #   donor_id: str, either the ID of the donor as specified in the donor_ids.tsv file, or 'unassigned'
    #   prob_donor: float, the probability that the cell is from the donor specified in donor_id
    #   prob_doublet: probability that the droplet represents a doublet, as determined by vireo
    #   prob_singlet: 1-prob_doublet
    # and ALL COLUMNS IN `expt_df` (this should always include 'library')

    umi_dgc_rownames = rownames(umi_dgc)
    umi_dgc_colnames = colnames(umi_dgc)

    # only consider cells that are in the filtered umi data
    mol_df = mol_df[mol_df$barcode %in% colnames(umi_dgc),]
    mol_df_grouped = mol_df %>% group_by(barcode) %>% summarize(nUmi=n(), nRead=sum(count), pct_intronic=sum(umi_type==0)/nUmi)
    rownames(mol_df_grouped) = mol_df_grouped$barcode

    # match order of other matrices / dfs to barcodes of umi_dgc 
    mol_df_grouped_matched = mol_df_grouped[umi_dgc_colnames, ]

    rownames(donor_ids) = donor_ids$cell
    donor_ids_matched = donor_ids[umi_dgc_colnames,]

    # sum within cells
    nGene = colSums(umi_dgc > 0)

    meta_df = data.frame(
        mol_df_grouped_matched$nUmi,
        nGene, 
        mol_df_grouped_matched$nRead,
        mol_df_grouped_matched$pct_intronic,
        donor_ids_matched$donor_id, # note that donor_ids_matched$donor_id is sometimes 'unassigned', even with high prob_max...
        donor_ids_matched$prob_max,
        1-donor_ids_matched$prob_doublet,
        donor_ids_matched$prob_doublet
    )

    colnames(meta_df) = c(
        "nUMI", "nGene", 'nRead', 'pct_intronic', 'donor_id', 'prob_donor', 'prob_singlet', 'prob_doublet')
    for (cname in colnames(expt_df)){
        meta_df[[cname]] = expt_df[[cname]]
    }
    rownames(meta_df) = colnames(umi_dgc)
    return(meta_df)
}     

sum_duplicate_rownames_of_dgc_matrix=function(dgc){   
    # some umi_data has repeated rownames which causes problems down the road
    # we will sum the umi counts in these rows
    # collect rows with duplicate rownames as a dataframe
    n_occur = data.frame(table(rownames(dgc)))
    repeated_rownames = n_occur[n_occur$Freq > 1,]$Var1
    duplicated_dgc = as.data.frame(dgc[rownames(dgc) %in% repeated_rownames,])
    duplicated_dgc['symbol'] = sub("\\.\\d$", "", rownames(duplicated_dgc))

    # sum across numeric columns
    numeric_cols = setdiff(colnames(duplicated_dgc), "symbol")
    summed_duped_umi_rows = duplicated_dgc %>%
        group_by(symbol) %>%
        summarise_at(vars(numeric_cols), sum, na.rm = TRUE) %>%
        ungroup() %>%
        as.data.frame()
    # set rownames to be symbol column, then remove it
    rownames(summed_duped_umi_rows) = summed_duped_umi_rows$symbol
    summed_duped_umi_rows = summed_duped_umi_rows[, !(names(summed_duped_umi_rows) %in% c('symbol'))]

    # Hack
    # each round of the loop will remove ONE of each duplicate row 
    for(i in 1:max(n_occur$Freq)){
        dgc = dgc[!rownames(dgc) %in% repeated_rownames,]
    }

    # finally, replace the deduped rows with the summed rows via an rbind
    dgc = rbind(dgc, as(summed_duped_umi_rows, "sparseMatrix"))

    return(dgc)
}



In [ ]:
# Apply the function to each directory in DIRS_TO_READ
DIRS_TO_READ = sapply(DIRS_TO_READ_LONG, extract_text)
names(DIRS_TO_READ_LONG) = DIRS_TO_READ

length(DIRS_TO_READ)

In [ ]:

GTEX_DIRS_TO_READ = names(DIRS_TO_READ_LONG)[DIRS_TO_READ_LONG %in% GTEX_DIRS_LONG]

# get dirs in GTEX_DIRS_TO_READ that do NOT contain /mnt/accessory/seq_data/gtex/vireo_outs/donor_list/donor_ids.tsv
skipped = c()
for (d in GTEX_DIRS_TO_READ){
    if (d == ""){next}
    donor_ids_path = file.path(BASE_PATH, "gtex", d, "vireo_outs", "donor_list", "donor_ids.tsv")
    if (!file.exists(donor_ids_path)){
        skipped = c(skipped, d)
    }
}
length(skipped)
skipped

In [ ]:
filtered_dgc_list = list()
sce_list = list()
raw_dgc_list = list()
raw_cd_list = list()
mol_info_list = list()
vireo_donor_ids_list = list()

expt_df = read.table(SUPPLEMENTARY_METADATA_PATH, sep="\t", header=TRUE)
expt_df = expt_df[,
    c("library", "library_long", "source", "sort", "region", "flowcell")
]

for (long_name in names(DIRS_TO_READ)){
    print("::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::")
    
    if(long_name == ''){
        next
    }

    name = DIRS_TO_READ[[long_name]]
    if (long_name %in% CALICO_DIRS_LONG){
        this_base_path = file.path(BASE_PATH, 'calico')
    } else if (long_name %in% GTEX_DIRS_LONG){
        this_base_path = file.path(BASE_PATH, 'gtex')
    }
    
    if (name %in% skipped){
        next
    }
    
    print(name)

    raw_counts_path = file.path(this_base_path, name, RAW_COUNTS_BASENAME)
    filtered_counts_path = file.path(this_base_path, name, FILTERED_COUNTS_BASENAME)
    molecule_info_path = file.path(this_base_path, name, MOLECULE_INFO_BASENAME)
    vireo_donor_ids_path = file.path(this_base_path, name, VIREO_DONOR_IDS_BASENAME)

    vireo_donor_ids = read.table(vireo_donor_ids_path, sep="\t", header=TRUE)

    print('CREATING DATA OBJECT DIRECTORY')
    dir.create(file.path(this_base_path, name, DATA_ING_DIRNAME), showWarnings = FALSE)

    print('READING FILTERED DGC .H5')
    # always read this
    filtered_dgc = Read10X_h5(filtered_counts_path)
    filtered_dgc_list[[name]] = filtered_dgc 

    if (MAKE_AND_SAVE_FILTERED_SCE_RDS){
    
        print("FETCHING MOLECULE INFO FROM .H5")
        # create a molecular data dataframe my fetching specific fields from the h5 and grouping by barcode
        mol_h5_file = file.path(molecule_info_path)
        h5fetch = function(x){return(h5read(mol_h5_file, x))}
        mol_df = data.frame(
            barcode=h5fetch("barcodes")[h5fetch("barcode_idx")+1] %>% paste0("-1"),
            feature=h5fetch("features/name")[h5fetch("feature_idx")+1],
            umi=h5fetch("umi"),
            umi_type=h5fetch("umi_type"),
            count=h5fetch("count")
        )
        
        if (SAVE_MOLECULE_INFO_RDS){
            print("SAVING MOLECULE INFO .RDS")
            saveRDS(mol_df, file=file.path(this_base_path, name, DATA_ING_DIRNAME, "molecule_info.rds"))
        }

        print("COLLECTING METADATA")
        # only consider cells that are in the filtered umi data
        mol_df = mol_df[mol_df$barcode %in% colnames(filtered_dgc),]

        vireo_donor_ids_list[[name]] = vireo_donor_ids

        expt_df_subset = expt_df[expt_df$library == name,]

        meta_df = createMetaDataFromDGCMatrix(
            expt_df_subset,
            filtered_dgc, 
            mol_df,
            vireo_donor_ids
        )

        print("BUILDING FILTERED SCE")
        data_sce=.myExpSetCreatorFn(inputExpData=filtered_dgc,
                        organism=ORGANISM,
                        minExpCells=MIN_EXP_CELLS,
                        inputPdata=meta_df,
                        inputFdata=NULL,
                        addExtraAnno=T,
                        ncores=12)
        
        print("SAVING FILTERED SCE .RDS")
        sce_list[[name]] = data_sce
        saveRDS(data_sce, file.path(this_base_path, name, DATA_ING_DIRNAME, "data_sce.rds"))
    }

    if (SAVE_RAW_RDS){
        print("READING RAW DGC .H5")
        raw_counts = Read10X_h5(raw_counts_path)
        raw_dgc_list[[name]] = raw_counts

        cd = data.frame(
            row.names=colnames(raw_counts),
            nUMI=colSums(raw_counts),
            log10_nUMI = log10(colSums(raw_counts) + 1),
            nGene=colSums(raw_counts > 0),
            is_in_filtered=colnames(raw_counts) %in% colnames(filtered_dgc_list[[name]])
        )
        raw_cd_list[[name]] = cd

        print("SAVING RAW DGC COL DATA RDS")
        saveRDS(cd, file.path(this_base_path, name, DATA_ING_DIRNAME, 'raw_feature_bc_matrix_col_data.rds'))
        print("SAVING RAW DGC RDS")
        saveRDS(raw_counts, file.path(this_base_path, name, DATA_ING_DIRNAME, 'raw_feature_bc_matrix.rds'))
    }
    print(paste0("DONE WITH LIBRARY", name))
    print("::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::")
}

In [ ]:
# CELLBENDER RDS MAKER
for (name in DIRS_TO_READ){
    print("::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::")
    
    if (name %in% skipped | name ==""){
        next
    }

    long_name = DIRS_TO_READ_LONG[[name]]

    if (long_name %in% CALICO_DIRS_LONG){
        this_base_path = file.path(BASE_PATH, 'calico')
    } else if (long_name %in% GTEX_DIRS_LONG){
        this_base_path = file.path(BASE_PATH, 'gtex')
    }    
    
    print(name)

    long_name = DIRS_TO_READ_LONG[[name]]

    if (long_name %in% CALICO_DIRS_LONG){
        this_base_path = file.path(BASE_PATH, 'calico')
    } else if (long_name %in% GTEX_DIRS_LONG){
        this_base_path = file.path(BASE_PATH, 'gtex')
    }

    orig_sce = readRDS(file.path(this_base_path, name, DATA_ING_DIRNAME, "data_sce.rds"))
    filtered_cb_path = file.path(this_base_path, name, "cb_outs", paste0(name, "_out_filtered.h5"))

    file_path = filtered_cb_path
    h5_data = h5read(file_path, "/matrix")
    counts = h5_data$data
    indices = h5_data$indices
    indptr = h5_data$indptr
    barcodes = h5_data$barcodes
    num_genes = length(h5_data$features$name)  # Number of genes
    num_cells = length(barcodes)  # Number of cells

    counts_matrix = sparseMatrix(i=indices + 1,  # +1 for 1-based indexing in R
                                    p=indptr,
                                    x=counts,
                                    dims=c(num_genes, num_cells))

    rownames(counts_matrix) = h5_data$features$name
    colnames(counts_matrix) = barcodes
    sce_orig = orig_sce
    counts_matrix_filtered = counts_matrix[, colnames(counts_matrix) %in% colnames(sce_orig)]
    counts_matrix_filtered = sum_duplicate_rownames_of_dgc_matrix(counts_matrix_filtered)

    cd = colData(sce_orig)
    cd = cd[rownames(cd) %in% colnames(counts_matrix_filtered), ]

    non_qc_cols <- colnames(cd)[!grepl("^QC", colnames(cd))]
    cd = cd[, non_qc_cols]

    h5fetch = function(x){return(h5read(filtered_cb_path, x))}
    cb_cd_supp = data.frame(
            barcodes = h5fetch('/matrix/barcodes'),
            cell_probability =  h5fetch('droplet_latents/cell_probability'),
            background_fraction = h5fetch('droplet_latents/background_fraction')
        )
    rownames(cb_cd_supp) = cb_cd_supp$barcodes
    cb_cd_supp = cb_cd_supp[rownames(cd), ]
    cb_cd_supp = cb_cd_supp[match(rownames(cd), rownames(cb_cd_supp)),]
    cd = cbind(cd, cb_cd_supp)

    cb_sce=.myExpSetCreatorFn(
            inputExpData=counts_matrix_filtered,
            organism=ORGANISM,
            minExpCells=MIN_EXP_CELLS,
            inputPdata=cd,
            inputFdata=NULL,
            addExtraAnno=T,
            ncores=12
        )
    
    print("SAVING CELLBENDER SCE AS .RDS")
    cb_rds_basename = paste0("cb_data_sce_FPR_", FPR, ".rds")
    saveRDS(cb_sce, file.path(this_base_path, name, DATA_ING_DIRNAME, cb_rds_basename))

    print(paste0("DONE WITH LIBRARY", name))
    print(":::::::::::::::::::::::::::::::::::::::::::::")
}